In [17]:
import pandas as pd
from pinecone import Pinecone, PodSpec
from dotenv import load_dotenv

In [18]:
load_dotenv(dotenv_path='../.env')

True

In [19]:
df = pd.read_csv('../datasets/medium_post_titles.csv')

In [20]:
df = df.dropna()

In [21]:
df = df[~df['subtitle_truncated_flag']]

In [22]:
df['title_extended'] = df['title'] + ' ' + df['subtitle']

In [24]:
pc = Pinecone()

In [26]:
pc.create_index(
    name='medium-articles',
    dimension=384,
    metric='cosine',
    spec=PodSpec(
        environment='gcp-starter'
    )
)

In [28]:
from sentence_transformers import SentenceTransformer
import torch

In [29]:
model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')

modules.json: 100%|██████████| 349/349 [00:00<00:00, 349kB/s]
c:\Users\Prithviraj_K_suvarna\Anaconda3\envs\langchain\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Prithviraj_K_suvarna\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(mes

In [36]:
df['values'] = df['title_extended'].map(
    lambda x: model.encode(x).tolist()
)

In [39]:
df['id'] = df.reset_index(drop = 'index').index

In [40]:
df['metadata']  = df.apply(lambda x : {
    'title': x['title'],
    'subtitle': x['subtitle'],
    'category': x['category']
}, axis=1)

In [47]:
df_upsert = df[['id', 'values', 'metadata']]
df_upsert['id'] = df_upsert['id'].astype(str)

C:\Users\Prithviraj_K_suvarna\AppData\Local\Temp\ipykernel_4996\410941487.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_upsert['id'] = df_upsert['id'].astype(str)


In [48]:
idx = pc.Index('medium-articles')

In [50]:
idx.upsert_from_dataframe(df_upsert)















sending upsert requests:   0%|          | 0/81545 [17:48<?, ?it/s]























































































































































sending upsert requests: 100%|██████████| 81545/81545 [07:38<00:00, 177.91it/s]


{'upserted_count': 81545}